# Event-to-Video Reconstruction with evlib

This notebook demonstrates the event-to-video reconstruction capabilities in evlib, including:

- Single frame reconstruction
- Multi-frame video generation
- Temporal binning effects
- Performance benchmarking
- Quality metrics analysis

## What is Event-to-Video Reconstruction?

Event-to-video reconstruction converts sparse event data from event cameras into dense intensity images. This is useful for:
- Visualization of event data
- Integration with traditional computer vision algorithms
- High-speed imaging applications
- Low-light and high-dynamic-range scenarios

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from pathlib import Path

# Import evlib
try:
    import evlib
    print(f"evlib version: {evlib.__version__}")
    print("Available modules:", [m for m in dir(evlib) if not m.startswith('_')])
except ImportError as e:
    print("Please install evlib: pip install evlib")
    raise e

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## 1. Generate Synthetic Event Data

First, let's create some synthetic event data that simulates a moving pattern.

In [2]:
def generate_synthetic_events(width=128, height=128, num_events=10000, pattern='moving_circle'):
    """Generate synthetic event data with different patterns."""
    events = []
    
    if pattern == 'moving_circle':
        # Moving circle pattern
        center_x = width // 2
        center_y = height // 2
        radius = 20
        
        for i in range(num_events):
            t = i * 0.0001  # 0.1ms intervals
            
            # Circle moves in a figure-8 pattern
            offset_x = 30 * np.sin(t * 10)
            offset_y = 15 * np.sin(t * 20)
            
            # Generate events on the circle perimeter
            angle = np.random.uniform(0, 2 * np.pi)
            x = int(center_x + offset_x + radius * np.cos(angle))
            y = int(center_y + offset_y + radius * np.sin(angle))
            
            # Add some noise
            x += np.random.randint(-2, 3)
            y += np.random.randint(-2, 3)
            
            polarity = 1 if np.random.random() > 0.5 else -1
            
            if 0 <= x < width and 0 <= y < height:
                events.append((x, y, t, polarity))
    
    elif pattern == 'rotating_bar':
        # Rotating bar pattern
        center_x = width // 2
        center_y = height // 2
        bar_length = 40
        
        for i in range(num_events):
            t = i * 0.0001
            
            # Bar rotates around center
            angle = t * 5  # Rotation speed
            
            # Generate events along the bar
            pos_along_bar = np.random.uniform(-bar_length/2, bar_length/2)
            x = int(center_x + pos_along_bar * np.cos(angle))
            y = int(center_y + pos_along_bar * np.sin(angle))
            
            polarity = 1 if np.random.random() > 0.3 else -1
            
            if 0 <= x < width and 0 <= y < height:
                events.append((x, y, t, polarity))
    
    return events

# Generate different patterns
print("Generating synthetic event data...")
circle_events = generate_synthetic_events(128, 128, 15000, 'moving_circle')
bar_events = generate_synthetic_events(128, 128, 12000, 'rotating_bar')

print(f"Generated {len(circle_events)} circle events")
print(f"Generated {len(bar_events)} bar events")

# Visualize event distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Circle events
circle_x = [e[0] for e in circle_events]
circle_y = [e[1] for e in circle_events]
circle_t = [e[2] for e in circle_events]
circle_p = [e[3] for e in circle_events]

axes[0].scatter(circle_x, circle_y, c=circle_p, s=0.5, cmap='RdBu', alpha=0.6)
axes[0].set_title('Moving Circle Events (Spatial)')
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')
axes[0].invert_yaxis()

# Bar events
bar_x = [e[0] for e in bar_events]
bar_y = [e[1] for e in bar_events]
bar_t = [e[2] for e in bar_events]
bar_p = [e[3] for e in bar_events]

axes[1].scatter(bar_x, bar_y, c=bar_p, s=0.5, cmap='RdBu', alpha=0.6)
axes[1].set_title('Rotating Bar Events (Spatial)')
axes[1].set_xlabel('X')
axes[1].set_ylabel('Y')
axes[1].invert_yaxis()

# Temporal distribution
axes[2].hist(circle_t, bins=50, alpha=0.7, label='Circle', density=True)
axes[2].hist(bar_t, bins=50, alpha=0.7, label='Bar', density=True)
axes[2].set_title('Event Temporal Distribution')
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('Density')
axes[2].legend()

plt.tight_layout()
plt.show()

In [ ]:
def events_to_evlib_format(events):
    """Convert event list to evlib numpy arrays."""
    if not events:
        return np.array([], dtype=np.int64), np.array([], dtype=np.int64), np.array([], dtype=np.float64), np.array([], dtype=np.int64)
    
    xs = np.array([e[0] for e in events], dtype=np.int64)
    ys = np.array([e[1] for e in events], dtype=np.int64)
    ts = np.array([e[2] for e in events], dtype=np.float64)
    ps = np.array([e[3] for e in events], dtype=np.int64)
    
    return xs, ys, ts, ps

# Convert event data to evlib format
print("Converting events to evlib format...")
circle_xs, circle_ys, circle_ts, circle_ps = events_to_evlib_format(circle_events)
bar_xs, bar_ys, bar_ts, bar_ps = events_to_evlib_format(bar_events)

print(f"Circle events: {len(circle_xs)} events")
print(f"  X range: [{circle_xs.min()}, {circle_xs.max()}]")
print(f"  Y range: [{circle_ys.min()}, {circle_ys.max()}]")
print(f"  Time range: [{circle_ts.min():.4f}, {circle_ts.max():.4f}]")
print(f"  Polarities: {np.unique(circle_ps)}")

print(f"\nBar events: {len(bar_xs)} events")
print(f"  X range: [{bar_xs.min()}, {bar_xs.max()}]")
print(f"  Y range: [{bar_ys.min()}, {bar_ys.max()}]")
print(f"  Time range: [{bar_ts.min():.4f}, {bar_ts.max():.4f}]")
print(f"  Polarities: {np.unique(bar_ps)}")

# Define reconstruction configurations
configs = {
    "Standard": {"num_bins": 5},
    "High Detail": {"num_bins": 8},
    "Fast": {"num_bins": 3}
}

print(f"\nAvailable reconstruction configurations:")
for name, config in configs.items():
    print(f"  {name}: {config}")

In [ ]:
# Perform reconstructions using evlib.processing.events_to_video
print("Performing event-to-video reconstructions...")

reconstruction_results = {}
timing_results = {}

for config_name, config in configs.items():
    print(f"\nTesting {config_name} configuration:")
    
    # Circle events
    start_time = time.time()
    circle_result = evlib.processing.events_to_video(
        circle_xs, circle_ys, circle_ts, circle_ps,
        height=128, width=128,
        num_bins=config['num_bins']
    )
    circle_time = time.time() - start_time
    
    # Bar events
    start_time = time.time()
    bar_result = evlib.processing.events_to_video(
        bar_xs, bar_ys, bar_ts, bar_ps,
        height=128, width=128,
        num_bins=config['num_bins']
    )
    bar_time = time.time() - start_time
    
    # Extract 2D frames if result is 3D
    if circle_result.ndim == 3:
        circle_result = circle_result[:, :, 0]
    if bar_result.ndim == 3:
        bar_result = bar_result[:, :, 0]
    
    reconstruction_results[config_name] = {
        'circle': circle_result,
        'bar': bar_result
    }
    
    timing_results[config_name] = {
        'circle_time': circle_time,
        'bar_time': bar_time
    }
    
    print(f"  Circle reconstruction: {circle_time:.3f}s")
    print(f"  Bar reconstruction: {bar_time:.3f}s")
    print(f"  Circle result shape: {circle_result.shape}")
    print(f"  Circle intensity range: [{np.min(circle_result):.3f}, {np.max(circle_result):.3f}]")

print("\nEvent-to-video reconstructions complete!")

## 3. Single Frame Reconstruction

Let's test event-to-video reconstruction with different temporal bin configurations.

In [ ]:
# Multi-frame video reconstruction
print("Testing multi-frame video reconstruction...")

# Reconstruct multiple frames from the circle events
num_frames = 10
start_time = time.time()
circle_frames = evlib.processing.reconstruct_events_to_frames(
    circle_xs, circle_ys, circle_ts, circle_ps,
    height=128, width=128,
    num_frames=num_frames,
    num_bins=5
)
multi_frame_time = time.time() - start_time

# Convert list of frames to numpy array for easier handling
circle_frames_array = np.array([frame[:, :, 0] for frame in circle_frames])

print(f"Multi-frame reconstruction: {multi_frame_time:.3f}s")
print(f"Number of frames: {len(circle_frames)}")
print(f"Frame shape: {circle_frames[0].shape}")
print(f"Frames per second: {num_frames/multi_frame_time:.1f}")

# Visualize frame sequence
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i in range(min(10, len(circle_frames))):
    # Extract the 2D frame from the (H, W, 1) array
    frame = circle_frames[i][:, :, 0]
    axes[i].imshow(frame, cmap='gray', vmin=0, vmax=1)
    axes[i].set_title(f'Frame {i}')
    axes[i].axis('off')

plt.suptitle(f'Multi-Frame Reconstruction ({num_frames} frames in {multi_frame_time:.2f}s)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Compare different num_bins settings on the same data
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Reconstruction with different bin counts
num_bins_values = [3, 5, 7]
bin_results = {}

for i, num_bins in enumerate(num_bins_values):
    # Circle
    circle_result = evlib.processing.events_to_video(
        circle_xs, circle_ys, circle_ts, circle_ps,
        height=128, width=128,
        num_bins=num_bins
    )
    # Extract 2D frame if result is 3D
    if circle_result.ndim == 3:
        circle_result = circle_result[:, :, 0]
    
    axes[0, i].imshow(circle_result, cmap='gray', vmin=0, vmax=1)
    axes[0, i].set_title(f'Circle - {num_bins} bins')
    axes[0, i].axis('off')
    
    # Bar
    bar_result = evlib.processing.events_to_video(
        bar_xs, bar_ys, bar_ts, bar_ps,
        height=128, width=128,
        num_bins=num_bins
    )
    # Extract 2D frame if result is 3D
    if bar_result.ndim == 3:
        bar_result = bar_result[:, :, 0]
        
    axes[1, i].imshow(bar_result, cmap='gray', vmin=0, vmax=1)
    axes[1, i].set_title(f'Bar - {num_bins} bins')
    axes[1, i].axis('off')
    
    bin_results[num_bins] = {'circle': circle_result, 'bar': bar_result}

plt.suptitle('Effect of Temporal Bins on Reconstruction', fontsize=16)
plt.tight_layout()
plt.show()

In [6]:
# Visualize reconstruction results
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

config_names = list(configs.keys())

# Circle reconstructions
for i, config_name in enumerate(config_names):
    result = reconstruction_results[config_name]['circle']
    axes[0, i].imshow(result, cmap='gray', vmin=0, vmax=1)
    axes[0, i].set_title(f'Circle - {config_name}\n({timing_results[config_name]["circle_time"]:.3f}s)')
    axes[0, i].axis('off')

# Bar reconstructions
for i, config_name in enumerate(config_names):
    result = reconstruction_results[config_name]['bar']
    axes[1, i].imshow(result, cmap='gray', vmin=0, vmax=1)
    axes[1, i].set_title(f'Bar - {config_name}\n({timing_results[config_name]["bar_time"]:.3f}s)')
    axes[1, i].axis('off')

plt.suptitle('E2VID Event-to-Video Results', fontsize=16)
plt.tight_layout()
plt.show()

# Show intensity histograms
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, config_name in enumerate(config_names):
    circle_result = reconstruction_results[config_name]['circle']
    bar_result = reconstruction_results[config_name]['bar']
    
    axes[i].hist(circle_result.flatten(), bins=50, alpha=0.7, label='Circle', density=True)
    axes[i].hist(bar_result.flatten(), bins=50, alpha=0.7, label='Bar', density=True)
    axes[i].set_title(f'{config_name} - Intensity Distribution')
    axes[i].set_xlabel('Intensity')
    axes[i].set_ylabel('Density')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Multi-Frame Video Reconstruction

Now let's test the multi-frame reconstruction functionality to create video sequences from events.

In [ ]:
# Multi-frame video reconstruction
print("Testing multi-frame video reconstruction...")

# Reconstruct multiple frames from the circle events
num_frames = 10
start_time = time.time()
circle_frames = evlib.processing.reconstruct_events_to_frames(
    circle_xs, circle_ys, circle_ts, circle_ps,
    height=128, width=128,
    num_frames=num_frames,
    num_bins=5
)
multi_frame_time = time.time() - start_time

print(f"Multi-frame reconstruction: {multi_frame_time:.3f}s")
print(f"Number of frames: {len(circle_frames)}")
print(f"Frame shape: {circle_frames[0].shape}")
print(f"Frames per second: {num_frames/multi_frame_time:.1f}")

# Visualize frame sequence
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i in range(min(10, len(circle_frames))):
    # Extract the 2D frame from the (H, W, 1) array
    frame = circle_frames[i][:, :, 0] if circle_frames[i].ndim == 3 else circle_frames[i]
    axes[i].imshow(frame, cmap='gray', vmin=0, vmax=1)
    axes[i].set_title(f'Frame {i}')
    axes[i].axis('off')

plt.suptitle(f'Multi-Frame Reconstruction ({num_frames} frames in {multi_frame_time:.2f}s)', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Temporal Bin Analysis

Let's analyze how the number of temporal bins affects reconstruction quality.

In [8]:
# Compare different num_bins settings on the same data
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Reconstruction with different bin counts
num_bins_values = [3, 5, 7]
bin_results = {}

for i, num_bins in enumerate(num_bins_values):
    # Circle
    circle_result = evlib.processing.events_to_video(
        circle_xs, circle_ys, circle_ts, circle_ps,
        height=128, width=128,
        num_bins=num_bins
    )
    axes[0, i].imshow(circle_result, cmap='gray', vmin=0, vmax=1)
    axes[0, i].set_title(f'Circle - {num_bins} bins')
    axes[0, i].axis('off')
    
    # Bar
    bar_result = evlib.processing.events_to_video(
        bar_xs, bar_ys, bar_ts, bar_ps,
        height=128, width=128,
        num_bins=num_bins
    )
    axes[1, i].imshow(bar_result, cmap='gray', vmin=0, vmax=1)
    axes[1, i].set_title(f'Bar - {num_bins} bins')
    axes[1, i].axis('off')
    
    bin_results[num_bins] = {'circle': circle_result, 'bar': bar_result}

plt.suptitle('Effect of Temporal Bins on Reconstruction', fontsize=16)
plt.tight_layout()
plt.show()

## 6. Performance Analysis

Let's analyze the performance characteristics of the reconstruction methods.

In [9]:
# Performance benchmarking with different event counts
def benchmark_reconstruction(event_pattern='moving_circle', event_counts=[1000, 5000, 10000, 20000]):
    """Benchmark reconstruction performance with different event counts."""
    single_frame_times = []
    multi_frame_times = []
    
    for count in event_counts:
        print(f"Benchmarking with {count} events...")
        
        # Generate events
        events = generate_synthetic_events(128, 128, count, event_pattern)
        xs, ys, ts, ps = events_to_evlib_format(events)
        
        # Benchmark single frame reconstruction
        start_time = time.time()
        evlib.processing.events_to_video(xs, ys, ts, ps, height=128, width=128, num_bins=5)
        single_time = time.time() - start_time
        single_frame_times.append(single_time)
        
        # Benchmark multi-frame reconstruction (5 frames)
        start_time = time.time()
        evlib.processing.reconstruct_events_to_frames(
            xs, ys, ts, ps, height=128, width=128, num_frames=5, num_bins=5
        )
        multi_time = time.time() - start_time
        multi_frame_times.append(multi_time)
        
        print(f"  Single frame: {single_time:.4f}s, Multi-frame (5): {multi_time:.4f}s")
    
    return event_counts, single_frame_times, multi_frame_times

# Run benchmark
print("Running performance benchmark...")
event_counts, single_times, multi_times = benchmark_reconstruction()

# Plot performance results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Absolute timing
ax1.plot(event_counts, single_times, 'o-', label='Single Frame', linewidth=2, markersize=8)
ax1.plot(event_counts, multi_times, 's-', label='Multi-Frame (5)', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Events')
ax1.set_ylabel('Reconstruction Time (s)')
ax1.set_title('Reconstruction Performance')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Throughput (events per second)
single_throughput = [count/time for count, time in zip(event_counts, single_times)]
multi_throughput = [count/(time/5) for count, time in zip(event_counts, multi_times)]  # Per frame

ax2.plot(event_counts, single_throughput, 'o-', label='Single Frame', linewidth=2, markersize=8)
ax2.plot(event_counts, multi_throughput, 's-', label='Multi-Frame (per frame)', linewidth=2, markersize=8)
ax2.set_xlabel('Number of Events')
ax2.set_ylabel('Throughput (Events/s)')
ax2.set_title('Reconstruction Throughput')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_yscale('log')

plt.tight_layout()
plt.show()

# Print performance summary
print("\nPerformance Summary:")
print(f"Single frame - Average throughput: {np.mean(single_throughput):.0f} events/s")
print(f"Multi-frame - Average throughput per frame: {np.mean(multi_throughput):.0f} events/s")
print(f"Multi-frame efficiency: {np.mean(multi_throughput)/np.mean(single_throughput):.1f}x")

## 7. Summary and Next Steps

This notebook demonstrated the event-to-video reconstruction capabilities in evlib:

In [10]:
print("="*60)
print("EVENT-TO-VIDEO RECONSTRUCTION SUMMARY")
print("="*60)

print("\n✓ Features Demonstrated:")
print("  • Event-to-video reconstruction API")
print("  • Single frame reconstruction")
print("  • Multi-frame video generation")
print("  • Temporal binning effects")
print("  • Performance benchmarking")
print("  • Synthetic and real data support")

print("\n📊 Key Findings:")
avg_single_time = np.mean(single_times)
avg_throughput = np.mean(single_throughput)

print(f"  • Single frame avg time: ~{avg_single_time:.3f}s")
print(f"  • Average throughput: ~{avg_throughput:.0f} events/s")
print(f"  • Multi-frame efficiency: ~{np.mean(multi_throughput)/np.mean(single_throughput):.1f}x")
print(f"  • Output intensity range: [0, 1]")
print(f"  • Higher num_bins → more temporal detail")
print(f"  • Configuration flexibility for different scenarios")

print("\n🚀 Next Steps:")
print("  1. Integrate ONNX Runtime for pre-trained models")
print("  2. Implement FireNet for speed optimization")
print("  3. Add GPU acceleration support")
print("  4. Develop quality assessment metrics")
print("  5. Create model conversion utilities")
print("  6. Add temporal consistency mechanisms")

print("\n💡 Usage Tips:")
print("  • Use events_to_video() for single frames")
print("  • Use reconstruct_events_to_frames() for videos")
print("  • Adjust num_bins based on motion characteristics")
print("  • Monitor performance vs quality trade-offs")
print("  • Ensure correct data types (int64 for x,y,p; float64 for t)")

print("\n" + "="*60)
print("For more examples, see: evlib/examples/")
print("For API documentation, see: https://github.com/tallamjr/evlib")
print("="*60)